In [1]:
import pandas as pd
import numpy as np
from source.Files_operating import read_sht_data, read_dataFile

shtRipper v1.3


In [4]:
F_ID = 44184
dir_path = "C:/Users/f.belous/Work/Projects/Plasma_analysis/data/sht/G-ELM/"  # D:/Edu/Lab/Projects/Plasma_analysis/data/sht/G-ELM/

df = read_sht_data(f'sht{F_ID}', dir_path)
df = df.rename(columns={"ch1": "d_alpha"})
df["sxr"] = read_sht_data(f'sht{F_ID}', dir_path, data_name="SXR 50 mkm").ch1
# dbs = read_dataFile(f'data/dbs/{F_ID} DBS.dat')
# mgd_data_1
# mgd_data_3
# mgd_data_2
df.describe()

,t,d_alpha,sxr
count,393216.000000,393216.000000,393216.000000
mean,0.196608,0.281714,0.386184
std,0.113512,0.393401,0.503047
min,0.000000,-0.388535,-0.076685
25%,0.098304,0.012781,0.107358
50%,0.196607,0.066460,0.109915
75%,0.294911,0.575134,0.439658
max,0.393215,4.818347,2.356772


In [76]:
d_alpha = df.d_alpha.to_numpy()
sxr = df.sxr.to_numpy()

In [94]:
d_alpha_d1 = np.diff(d_alpha)
sxr_d1 = np.diff(sxr)

In [95]:
from scipy import signal

b, a = signal.butter(5, 0.1)
d_alpha_f = signal.filtfilt(b, a, d_alpha_d1)
sxr_f = signal.filtfilt(b, a, sxr_d1)

In [101]:
da_f_max, da_f_std = d_alpha_f.max(), d_alpha_f.std()
sxr_f_max, sxr_f_std = sxr_f.max(), sxr_f.std()

In [206]:
times_da_init = df.t.to_numpy()[:-1][abs(d_alpha_f - np.quantile(d_alpha_f, 0.9)) > d_alpha_f.std() * 3]
times_sxr_init = df.t.to_numpy()[:-1][abs(sxr_f - np.quantile(sxr_f, 0.9)) > sxr_f.std() * 6]

print(da_f_max, da_f_std, times_da_init.shape)
print(sxr_f_max, sxr_f_std, times_sxr_init.shape)

0.23383257112357403 0.0025870038350616488 (2582,)
0.015201329800146266 0.0009320451522782375 (1080,)


In [207]:
count = 0
for i in range(times_da_init.shape[0]):
    if times_da_init[i] in times_sxr_init:
        count += 1

print(count)

111


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

mode_ploting = 1  # int(input("Input visualising mode [ 0 - manual | 1 - only marked ]:"))
# mode_marking = int(input("Input marking mode (0 - manual | 1 - semiauto): ").strip().split()[0])
l_edge = 184000  # int(input(f"Input start index [0:{df.shape[0]//1000*1000}]: ").strip().split()[0])
# print("---")
step = 2000
width = 3000

times_da = times_da_init[times_da_init * 1e6 >= l_edge]
times_sxr = times_sxr_init[times_sxr_init * 1e6 >= l_edge]

step_away = 10
time_ind = [0, 0]

while l_edge < df.shape[0]:
    r_edge = min(l_edge + width, df.shape[0] - 1)

    slice_da_times = []
    while time_ind[0] < times_da.shape[0] and l_edge <= int(times_da[time_ind[0]] * 1e6) <= r_edge:
        slice_da_times.append(times_da[time_ind[0]])
        time_ind[0] += 1
        
    slice_sxr_times = []
    while time_ind[1] < times_sxr.shape[0] and l_edge <= times_sxr[time_ind[1]] * 1e6 <= r_edge:
        slice_sxr_times.append(times_sxr[time_ind[1]])
        time_ind[1] += 1
        
    
    if mode_ploting == 1 and len(slice_da_times) == 0 and len(slice_sxr_times) == 0:
        l_edge += step
        continue
    
    fig, [ax1, ax2] = plt.subplots(2, 1)

    fig.set_figwidth(16)
    fig.set_figheight(8)

    ax1.plot(range(l_edge, r_edge), d_alpha[l_edge:r_edge], label="D-alpha")
    ax1.plot(range(l_edge, r_edge), d_alpha_d1[l_edge:r_edge], label="Diff 1")
    ax1.plot(range(l_edge, r_edge), d_alpha_f[l_edge:r_edge], label="Filter")
    for time in slice_da_times:
        ax1.axvline(int(time * 1e6), color="green", linestyle=':', linewidth=0.7)

    ax1.grid(which='major', color='#DDDDDD', linewidth=0.9)
    ax1.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(10))
    ax1.legend()

    ax2.plot(range(l_edge, r_edge), sxr[l_edge:r_edge], label="SXR")
    ax2.plot(range(l_edge, r_edge), sxr_d1[l_edge:r_edge], label="Diff 1")
    ax2.plot(range(l_edge, r_edge), sxr_f[l_edge:r_edge], label="Filter")
    for time in slice_sxr_times:
        ax2.axvline(int(time * 1e6), color="green", linestyle=':', linewidth=0.7)

    ax2.grid(which='major', color='#DDDDDD', linewidth=0.9)
    ax2.grid(which='minor', color='#DDDDDD', linestyle=':', linewidth=0.7)
    ax2.minorticks_on()
    ax2.xaxis.set_minor_locator(AutoMinorLocator(10))
    ax2.legend()
    plt.show()

    # res = list(map(int, input(f"Input index pairs of ELM fragments ({df.t[l_edge]} - {df.t[r_edge]} ms):\n").strip().split()))
    # for i in range(0, len(res), 2):
    #     board_ind = [res[i], res[i+1]]

    #     mark = float(input("Input mark to set [ 0 | 1 | 2 ]: "))
        
    #     if mode_marking and mark > 0:
    #         board_ind = get_borders(df.loc[res[i]:res[i + 1], "ch1"].to_numpy(), scale=1.5)
    #         board_ind[0] = max(board_ind[0] - step_away + res[i], 0)
    #         board_ind[1] += res[i]
        
    #     df.loc[board_ind[0]:board_ind[1], "ch1_marked"] = mark
    input(f"ELM fragment ({df.t[l_edge]} - {df.t[r_edge]} ms):\n")

    plt.close()
    plt.clf()
    l_edge += step

# 217500